In [1]:
# set the working directory for the project
%cd /home/vcap/app/cf-jupyterlab-workshop

/home/vcap/app/cf-jupyterlab-workshop


In [2]:
#uv add langgraph.prebuilt langgraph
import os
import requests
import json
from openai import OpenAI
import httpx
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain.tools import tool
from datetime import date
import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
import nest_asyncio
import ssl
from cfenv import AppEnv
import sys, os
from tanzu_utils import CFGenAIService

ModuleNotFoundError: No module named 'langchain_core.memory'

In [5]:
# Set environment variables to disable SSL verification
os.environ['PYTHONHTTPSVERIFY'] = '0'
os.environ['CURL_CA_BUNDLE'] = ''

# Create a custom SSL context that doesn't verify certificates
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

# Override the default SSL context
ssl._create_default_https_context = lambda: ssl_context

In [6]:
nest_asyncio.apply()
# Load CF environment
env = AppEnv()

# load your service details replace name with your Gen AI service.  Gen AI service is bound to the app
genai = CFGenAIService("gpt-oss:120b")
print("API Base:", genai.api_base)
print("Config URL:", genai.config_url)

# List available models
models = genai.list_models()
for m in models:
    print(f"- {m['name']} (capabilities: {', '.join(m['capabilities'])})")

# construct chat_credentials
chat_credentials = {
    "api_base": genai.api_base,
    "api_key": genai.api_key,
    "model_name": models[0]["name"]
}

# 2. HTTP client (optional but recommended for custom config)
httpx_client = httpx.Client(verify=False)  # verify=False if your endpoint needs --insecure

# 3. Initialize the LLM
llm = ChatOpenAI(
    temperature=0.9,
    model=chat_credentials["model_name"],   # model name from CF service
    base_url=chat_credentials["api_base"],  # OpenAI-compatible endpoint
    api_key=chat_credentials["api_key"],    # Bearer token
    http_client=httpx_client
)

API Base: https://genai-proxy.sys.tas-ndc.kuhn-labs.com/tanzu-gpt-oss-120b-b83126a/openai
Config URL: https://genai-proxy.sys.tas-ndc.kuhn-labs.com/tanzu-gpt-oss-120b-b83126a/config/v1/endpoint
- gpt-oss:120b (capabilities: CHAT, TOOLS)


/home/vcap/app/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'genai-proxy.sys.tas-ndc.kuhn-labs.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [7]:
# --- Time MCP service ---
time_mcp_service = env.get_service(name="time-mcp-server")
time_mcp_credentials = time_mcp_service.credentials
print(time_mcp_credentials["mcpServiceURL"])

time_mcp = time_mcp_credentials["mcpServiceURL"] + "/sse"

# --- Petclinic MCP service ---
petclinic_mcp_service = env.get_service(name="petclinic-mcp-server")
petclinic_mcp_credentials = petclinic_mcp_service.credentials


petclinic_mcp = petclinic_mcp_credentials["mcpServiceURL"] + "/sse"

# Async HTTP client for MCP
#async_client = httpx.AsyncClient(http2=True, verify=False, timeout=10.0)
# Patch httpx globally for MCP client
original_async_client = httpx.AsyncClient

class NoSSLAsyncClient(httpx.AsyncClient):
    def __init__(self, **kwargs):
         kwargs['verify'] = False
         super().__init__(**kwargs)

httpx.AsyncClient = NoSSLAsyncClient

# Create MCP client
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "sse",
            "url": time_mcp
        },
        "petclinic": {
            "transport": "sse",
            "url": petclinic_mcp
        }
    }
)
#Fetch and print tools
tools = await client.get_tools()
print("MCP Tools:", tools)


https://time-mcp-server.apps.tas-ndc.kuhn-labs.com
MCP Tools: [StructuredTool(name='getCurrentTime', description='Get the current time', args_schema={'type': 'object', 'properties': {}, 'required': [], 'additionalProperties': False}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x7f99cd9f6c00>), StructuredTool(name='fetch_petsByStatus', description='Retrieve pets from petstore api', args_schema={'properties': {'status': {'title': 'Status', 'type': 'string'}}, 'required': ['status'], 'title': 'fetch_petsByStatusArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x7f99cd9f7100>)]


In [9]:
# get current time
time_tool = [t for t in tools if t.name == "getCurrentTime"][0]
result = await time_tool.arun({})
print("Time tool result:", result)

# fetch pets
pet_tool = [t for t in tools if t.name == "fetch_petsByStatus"][0]
result = await pet_tool.arun({"status": "available"})
print("Pet tool result:", result)

Time tool result: [2025,9,19,18,9,42,187770903]
Pet tool result: ['{\n  "id": 7,\n  "category": {\n    "id": 4,\n    "name": "Lions"\n  },\n  "name": "Lion 1",\n  "photoUrls": [\n    "url1",\n    "url2"\n  ],\n  "tags": [\n    {\n      "id": 1,\n      "name": "tag1"\n    },\n    {\n      "id": 2,\n      "name": "tag2"\n    }\n  ],\n  "status": "available"\n}', '{\n  "id": 8,\n  "category": {\n    "id": 4,\n    "name": "Lions"\n  },\n  "name": "Lion 2",\n  "photoUrls": [\n    "url1",\n    "url2"\n  ],\n  "tags": [\n    {\n      "id": 1,\n      "name": "tag2"\n    },\n    {\n      "id": 2,\n      "name": "tag3"\n    }\n  ],\n  "status": "available"\n}', '{\n  "id": 234901432,\n  "name": "Shiro",\n  "photoUrls": [\n    "anyTestURLString"\n  ],\n  "tags": [],\n  "status": "available"\n}', '{\n  "id": 321,\n  "category": {\n    "id": 1,\n    "name": "Dogs"\n  },\n  "name": "TestPet",\n  "photoUrls": [\n    "https://example.com/photo1.jpg"\n  ],\n  "tags": [\n    {\n      "id": 1,\n      "na